In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Movshon lab - OpenEphys + Expo Converter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [2]:
from movshon_lab_to_nwb import MovshonOpenEphysNWBConverter
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import yaml
import pprint

## Step 1 - Converter.get_source_schema()

In [3]:
# Get source_schema
source_schema = MovshonOpenEphysNWBConverter.get_source_schema()
pprint.pprint(source_schema['properties'], width=120)

{'ExpoDataInterface': {'additionalProperties': False,
                       'properties': {'expo_file': {'description': 'path to xml file containing expo data',
                                                    'format': 'file',
                                                    'type': 'string'}},
                       'required': [],
                       'type': 'object'},
 'OpenEphysRecordingExtractorInterface': {'additionalProperties': False,
                                          'properties': {'folder_path': {'description': 'Path to directory containing '
                                                                                        'OpenEphys files.',
                                                                         'format': 'directory',
                                                                         'type': 'string'}},
                                          'required': ['folder_path'],
                                          'type': 'obj

## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [4]:
# Source data
base_path = Path('/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/expo/exampledata/expo_openephys/m666l3#7')

# oephys_path = Path('/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/oephys/')
oephys_dir = base_path / 'openephys/'
expo_file = base_path / 'm666l3#7[ori16].xml'

source_data = dict(
    OpenEphysRecordingExtractorInterface=dict(
        folder_path=str(oephys_dir),
    ),
    ExpoDataInterface=dict(
        expo_file=str(expo_file)
    )
)

pprint.pprint(source_data, width=120)

{'ExpoDataInterface': {'expo_file': '/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/expo/exampledata/expo_openephys/m666l3#7/m666l3#7[ori16].xml'},
 'OpenEphysRecordingExtractorInterface': {'folder_path': '/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/expo/exampledata/expo_openephys/m666l3#7/openephys'}}


## Step 3 - Instantiate Converter

In [5]:
# Initialize converter
converter = MovshonOpenEphysNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

Loading Open-Ephys: reading settings...
Decoding data from  openephys  format
Data interfaces for this converter:
{'ExpoDataInterface': <movshon_lab_to_nwb.expodatainterface.ExpoDataInterface object at 0x7fe5f49b4be0>,
 'OpenEphysRecordingExtractorInterface': <nwb_conversion_tools.datainterfaces.openephysdatainterface.OpenEphysRecordingExtractorInterface object at 0x7fe6090c9048>}


## Step 4 - Converter.get_metadata_schema()

In [6]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
pprint.pprint(metadata_schema, width=120)

{'$id': 'metadata.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': False,
 'description': 'Schema for the metadata',
 'properties': {'Ecephys': {'Device': {'additionalProperties': False,
                                       'properties': {'description': {'description': 'Description of the device (e.g., '
                                                                                     'model, firmware version, '
                                                                                     'processing software version, '
                                                                                     'etc.)',
                                                                      'type': 'string'},
                                                      'manufacturer': {'description': 'the name of the manufacturer of '
                                                                                      'this device',
            

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [7]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Rhythm FPGA'}], 'ElectrodeGroup': []},
 'NWBFile': {'identifier': 'f9b8ea1a-7784-11eb-be97-dd052e142a77',
             'session_description': 'no description',
             'session_start_time': datetime.datetime(2017, 8, 7, 21, 31, 32, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 6 - Get user-input metadata

In [8]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Rhythm FPGA'}], 'ElectrodeGroup': []},
 'NWBFile': {'identifier': 'f9b8ea1a-7784-11eb-be97-dd052e142a77',
             'session_description': 'example conversion',
             'session_start_time': datetime.datetime(2017, 8, 7, 21, 31, 32, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 7 - Converter.get_conversion_options_schema()

In [9]:
conversion_options_schema = converter.get_conversion_options_schema()

print('Conversion options for each data interface: \n')
pprint.pprint(conversion_options_schema['properties'], width=120)

Conversion options for each data interface: 

{'ExpoDataInterface': {'additionalProperties': False,
                       'properties': {'convert_expo': {'description': 'Whether to convert Expo data to NWB or not',
                                                       'type': 'boolean'}},
                       'required': [],
                       'type': 'object'},
 'OpenEphysRecordingExtractorInterface': {'additionalProperties': False,
                                          'properties': {'overwrite': {'default': False, 'type': 'boolean'},
                                                         'save_path': {'type': 'string'},
                                                         'stub_test': {'default': False, 'type': 'boolean'},
                                                         'use_timestamps': {'default': False, 'type': 'boolean'},
                                                         'write_as_lfp': {'default': False, 'type': 'boolean'}},
                   

## Step 8 - Get user-input conversion options

In [10]:
conversion_options = dict(
    OpenEphysRecordingExtractorInterface=dict(write_as_lfp=False),
    ExpoDataInterface=dict(convert_expo=True)
)

## Step 9 - Run conversion user filled metadata and conversion_options

In [11]:
output_file = 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,
    conversion_options=conversion_options
)

Adding expo data...
NWB file saved at out_example.nwb!


## Final 1 - Check NWB file

In [12]:
# load file
fname = 'out_example.nwb'
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

root pynwb.file.NWBFile at 0x140625628296920
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2021, 2, 25, 17, 17, 37, 602780, tzinfo=tzoffset(None, 3600))]
  identifier: f9b8ea1a-7784-11eb-be97-dd052e142a77
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  session_description: example conversion
  session_start_time: 2017-08-07 21:31:32-05:00
  timestamps_reference_time: 2017-08-07 21:31:32-05:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>



## Final 2 - Check NWB file with widgets

In [13]:
io = NWBHDF5IO(fname, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)